In [3]:
from bokeh.io import output_notebook
from bokeh.layouts import row
from bokeh.plotting import show

from src.libs.chart import (
    BarChart,
    BarChartConfig,
    ChartData,
    PieChart,
    PieChartConfig,
)

output_notebook()


Loading BokehJS ...

In [4]:
# 円グラフ設定
pie_config = PieChartConfig(
    title="Pizza Orders - Pie Chart",
    label_position_adjust=1.3,
)

# 棒グラフ設定
bar_config = BarChartConfig(
    title="Pizza Orders - Bar Chart",
)

# データ

data = ChartData(
    x=["Pepperoni", "Cheese", "Mixed Veggies", "Bacon"],
    y=[221, 212, 152, 72],
    colors=["red", "darkorange", "darkgreen", "hotpink"],
)


# 円グラフ
pie_chart = PieChart(data, pie_config)
pie_figure = pie_chart.render()

# 棒グラフ
bar_chart = BarChart(data, bar_config)
bar_figure = bar_chart.render()

# 並べて表示
show(row(pie_figure, bar_figure))

In [43]:
import math

import pandas as pd

df = pd.DataFrame(data.model_dump())

label_adjust_num = 1.3
total = df["y"].sum()

df = (
    df.assign(
        angle=lambda x: x["y"] / x["y"].sum() * 2 * math.pi,
        start_angle=lambda x: x["angle"].shift(1).fillna(0).cumsum() + math.pi / 2,
        end_angle=lambda x: x["start_angle"] + x["angle"],
    )
    .assign(
        middle_angle=lambda x: (x["start_angle"] + x["end_angle"]) / 2,
        label_x=lambda x: x["middle_angle"].apply(lambda angle: math.cos(angle) * label_adjust_num),
        label_y=lambda x: x["middle_angle"].apply(lambda angle: math.sin(angle) * label_adjust_num),
    )
    .assign(
        label_percentage=lambda x: x["y"] / x["y"].sum() * 100,
    )
    .pipe(lambda x: x.assign(label_amount=lambda x: x["y"].apply(lambda y: f"{y:.0f}pcs")))
    .pipe(
        lambda x: x.assign(
            label_percentage=lambda x: x["label_percentage"].apply(lambda y: f"{y:.2f}%")
        )
    )
)
df


,x,y,colors,angle,start_angle,end_angle,middle_angle,label_x,label_y,label_percentage,label_amount
0,Pepperoni,221,red,2.113522,1.570796,3.684318,2.627557,-1.131998,0.639204,33.64%,221pcs
1,Cheese,212,darkorange,2.027451,3.684318,5.711769,4.698044,-0.018648,-1.299866,32.27%,212pcs
2,Mixed Veggies,152,darkgreen,1.453644,5.711769,7.165413,6.438591,1.284333,0.201216,23.14%,152pcs
3,Bacon,72,hotpink,0.688568,7.165413,7.853982,7.509698,0.438780,1.223713,10.96%,72pcs


In [49]:
import math

import pandas as pd
import polars as pl

df_polars = pl.DataFrame(data.model_dump())

df_polars = df_polars.with_columns(
    [
        (pl.col("y") / total * 2 * math.pi).alias("angle"),
        (pl.col("angle").shift(1).fill_null(0).cumsum() + math.pi / 2).alias("start_angle"),
        (pl.col("start_angle") + pl.col("angle")).alias("end_angle"),
        ((pl.col("start_angle") + pl.col("end_angle")) / 2).alias("middle_angle"),
        (pl.col("middle_angle").apply(lambda angle: math.cos(angle) * label_adjust_num)).alias(
            "label_x"
        ),
        (pl.col("middle_angle").apply(lambda angle: math.sin(angle) * label_adjust_num)).alias(
            "label_y"
        ),
        (pl.col("y") / total * 100).alias("label_percentage"),
        (pl.col("y").apply(lambda y: f"{y:.0f}pcs")).alias("label_amount"),
        (pl.col("label_percentage").apply(lambda y: f"{y:.2f}%")).alias("label_percentage"),
    ]
)

df_polars

AttributeError: 'Expr' object has no attribute 'cumsum'